In [ ]:
import cv2
import numpy as np
import pandas as pd
import cvzone
from tracker import *
from ultralytics import YOLO

model = YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)


cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture(r'D:\Courses language programming\7_Computer Vision\Computer Vision - Full Course\6-Object Detection\Counting-people-in-a-marathon-using-YOLOv8-main\p3\p3.mp4')

my_file = open(r'D:\Courses language programming\7_Computer Vision\Computer Vision - Full Course\6-Object Detection\coco.txt', 'r')
data = my_file.read()
class_list = data.split('\n')


count = 0
tracker = Tracker()

cy1 = 383
offset = 4
counter = []

while True:
    ret, image = cap.read()
    if not ret: break
    count += 1
    if count % 3 != 0:
        continue

    image = cv2.resize(image, (1020, 500))

    result = model.predict(image)
    # print(result)
    a = result[0].boxes.data
    px = pd.DataFrame(a).astype(float)
    # print(px)

    list = []

    for index, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'person' in c:
            list.append([x1, y1, x2, y2])


    bbox_id = tracker.update(list)
    for box in bbox_id:
        x3, y3, x4, y4, id = box
        cx, cy = int(x3+x4) // 2, int(y3+y4) // 2

        cv2.putText(image, f'{id}', (x3, y3), cv2.FONT_ITALIC, 0.6,
                    (255, 255, 255), 1)
        cv2.rectangle(image, (x3, y3), (x4, y4), (255, 0, 0), 2)

        if (offset + cy) > cy1 > (cy - offset):
            if counter.count(id) == 0:
                counter.append(id)
                cv2.circle(image, (cx, cy), 4, (0, 255, 255), -1)


    cv2.line(image, (355, cy1), (696, cy1), (0, 255, 0), 2)
    u = (len(counter))
    cvzone.putTextRect(image, f'People count:- {u}', (50, 60), 2, 2)
    cv2.imshow('RGB', image)
    if cv2.waitKey(1) == ord('o'): break

cv2.destroyAllWindows()
cap.release()

In [ ]:
px